# Segment Tree

区間最小値 / 最大値などの区間に対するクエリに答えられる。<br>
区間内の値を効率的に更新できる。<br>
> https://ikatakos.com/pot/programming_algorithm/data_structure/segment_tree

## 計算量
- 前計算: $O(logN)$
- クエリ処理: $O(logN)$

## 実装

In [1]:
# セグメント木で管理する関数
# モノイドある必要がある
def segfunc(x, y):
    return x+y

class SegTree:
    # N: 要素数, INF: 初期値
    def __init__(self, segfunc, N: int, INF: int):
        # セグメント木の要素数
        n0 = 1 << ((N-1).bit_length())
        self.offset = n0
        self.data = [INF] * (n0 << 1)
        
        self.segfunc = segfunc
        self.INF = INF
    
    # インデックス pos の値を x に更新
    # pos: 0-indexed
    # x: 更新する値
    def update(self, pos, x):
        # 実際のデータは最下段のインデックス [n0,2*n0) に入る
        # pos に対応するセグメント木の要素のインデックスは pos + self.offset
        pos += self.offset
        # 更新処理
        self.data[pos] = x
        # 親を更新していく
        while pos > 1:
            # 左シフトして親のインデックスを求める
            pos >>= 1
            # pos の子に対して segfunc を適用
            self.data[pos] = self.segfunc(self.data[pos<<1], self.data[pos<<1 | 1])
    
    # 区間 [l,r) に segfunc を適用する処理
    # l, r: 0-indexed
    def query(self, l, r):
        result = self.INF
        # 最下段の要素のインデックス
        l = l + self.offset
        r = r + self.offset
        # 最下段の左と右の外側から、親を辿って範囲を狭めながら segfunc を適用していく
        while l < r:
            if l & 1:
                # 左端のブロックを調べる
                result = self.segfunc(result, self.data[l])
                # 右のブロックへ
                l += 1
            if r & 1:
                # 左のブロックを調べる
                result = self.segfunc(result, self.data[r-1])
            # 親ブロックへ移動
            l >>= 1
            r >>= 1
        
        return result

## [使用例 1](https://atcoder.jp/contests/tessoku-book/tasks/tessoku_book_bf)
長さ $N$ のある区間内の値を更新する or 最大値を答えるクエリに $Q$ 回答える。$O(NQ)$ だと TLE する。

In [2]:
def segfunc(x, y):
    return max(x,y)

N,Q=8,4
Queries=[[1, 3, 16], [2, 4, 7], [1, 5, 13], [2, 4, 7]]
sg = SegTree(segfunc, N, 0)

for q,a,b in Queries:
    a-=1
    if q==1:
        sg.update(a,b)
    else:
        b-=1
        res=sg.query(a,b)
        print(res)

0
13
